# 03 — Entrenamiento por Aprendizaje por Refuerzo (RL)

Este notebook describe el proceso de entrenamiento de los modelos de RL que forman parte
del sistema de trading. Se cubren:

1. Fundamentos de RL aplicado a trading
2. Algoritmo PPO (Proximal Policy Optimization)
3. Optimización de hiperparámetros con Optuna (Phase 3)
4. Entrenamiento 30-fold ensemble (Phase 4)
5. Prevención de sobreajuste: control de rollouts

> **Nota**: Se presenta la metodología general. Los detalles de la función de recompensa
> y la arquitectura exacta del agente son confidenciales.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

sns.set_theme(style='whitegrid', palette='muted')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['figure.dpi'] = 100

## 1. RL aplicado a trading

### ¿Por qué Aprendizaje por Refuerzo?

A diferencia de los enfoques supervisados (clasificación/regresión), el RL permite:
- **Decisiones secuenciales**: Cada acción afecta el estado futuro
- **Optimización del resultado final**: No predice precio, sino maximiza P&L
- **Gestión de posición**: Decide cuándo mantener, no solo cuándo entrar

### Formulación como MDP

| Componente | Descripción |
|------------|-------------|
| **Estado (s)** | Indicadores técnicos + información de posición actual |
| **Acción (a)** | HOLD / CLOSE (decisión binaria sobre posición abierta) |
| **Recompensa (r)** | Función que incentiva beneficio y penaliza riesgo |
| **Transición** | Avance al siguiente periodo temporal (determinista) |

### PPO vs SAC

Se evaluaron dos algoritmos:

| Algoritmo | Tipo | Resultado |
|-----------|------|----------|
| **PPO** (Proximal Policy Optimization) | On-policy | Resultados consistentes y reproducibles |
| **SAC** (Soft Actor-Critic) | Off-policy | Resultados no reproducibles, descartado |

PPO fue seleccionado por su **estabilidad** y **reproducibilidad**.

In [ ]:
# Visualización: convergencia típica de PPO en trading
np.random.seed(42)

# Simular curva de aprendizaje realista
timesteps = np.arange(0, 25000, 100)
base_reward = -5 + 15 * (1 - np.exp(-timesteps / 8000))
noise = np.random.normal(0, 2, len(timesteps))
episode_rewards = base_reward + noise

# Media móvil
window = 20
smooth_rewards = pd.Series(episode_rewards).rolling(window).mean()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Curva de aprendizaje
axes[0].scatter(timesteps, episode_rewards, alpha=0.15, s=5, color='steelblue')
axes[0].plot(timesteps, smooth_rewards, color='red', linewidth=2, label=f'Media móvil ({window} ep.)')
axes[0].axhline(0, color='gray', linestyle='--', alpha=0.5)
axes[0].set_xlabel('Timesteps', fontsize=11)
axes[0].set_ylabel('Recompensa por Episodio', fontsize=11)
axes[0].set_title('Curva de Aprendizaje PPO (ejemplo)', fontsize=12)
axes[0].legend()

# Distribución de recompensas: inicio vs final
early = episode_rewards[:50]
late = episode_rewards[-50:]
axes[1].hist(early, bins=20, alpha=0.6, color='lightcoral', label='Primeros 50 episodios')
axes[1].hist(late, bins=20, alpha=0.6, color='green', label='Últimos 50 episodios')
axes[1].set_xlabel('Recompensa', fontsize=11)
axes[1].set_ylabel('Frecuencia', fontsize=11)
axes[1].set_title('Distribución de Recompensas: Antes vs Después', fontsize=12)
axes[1].legend()

plt.tight_layout()
plt.show()

## 2. Prevención de sobreajuste: Control de rollouts

### El problema

Un **rollout** es un pase completo por los datos de entrenamiento.
Demasiados rollouts causan que el modelo **memorice** los datos en vez de aprender patrones.

```
Rollouts = Timesteps / Tamaño_datos
```

### Niveles de riesgo

| Rollouts | Riesgo | Impacto |
|----------|--------|---------|
| 2-3x | Bajo | Aprendizaje óptimo |
| 5-10x | Medio | Sobreajuste moderado |
| 10-40x | Alto | Sobreajuste severo |
| 40x+ | Crítico | Memorización total |

### Ejemplo real del proyecto

Con 12,367 velas de datos y timesteps fijos de 500,000:
- `500,000 / 12,367 = 40.4 rollouts` → **sobreajuste extremo**

La solución: `timesteps = tamaño_datos × 2` (máximo 3x)

In [ ]:
# Demostración: impacto del número de rollouts en sobreajuste
np.random.seed(42)

rollout_levels = [2, 5, 10, 20, 40]
train_pfs = [1.3, 1.5, 1.8, 2.5, 3.8]  # PF en training
test_pfs = [1.25, 1.15, 0.95, 0.72, 0.55]  # PF en test (datos nuevos)

fig, ax = plt.subplots(figsize=(10, 6))

x = np.arange(len(rollout_levels))
width = 0.35

bars1 = ax.bar(x - width/2, train_pfs, width, label='PF Training (in-sample)',
               color='steelblue', alpha=0.8)
bars2 = ax.bar(x + width/2, test_pfs, width, label='PF Test (out-of-sample)',
               color='crimson', alpha=0.8)

ax.axhline(1.0, color='red', linestyle='--', alpha=0.5, label='Breakeven (PF=1.0)')
ax.set_xlabel('Número de Rollouts', fontsize=12)
ax.set_ylabel('Profit Factor', fontsize=12)
ax.set_title('Sobreajuste: Training PF vs Test PF según rollouts', fontsize=13)
ax.set_xticks(x)
ax.set_xticklabels([f'{r}x' for r in rollout_levels])
ax.legend()

# Anotar la zona peligrosa
ax.annotate('Zona óptima\n(2-3x)', xy=(0, 1.3), fontsize=10, color='green',
            ha='center', va='bottom', fontweight='bold')
ax.annotate('¡Sobreajuste\nsevero!', xy=(4, 3.8), fontsize=10, color='red',
            ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

print('Degradación por rollouts:')
for rollouts, train_pf, test_pf in zip(rollout_levels, train_pfs, test_pfs):
    degradation = (train_pf - test_pf) / train_pf * 100
    status = 'OK' if degradation < 15 else 'SOBREAJUSTE'
    print(f'  {rollouts:2d}x: Train PF={train_pf:.2f}, Test PF={test_pf:.2f}, '
          f'Degradación={degradation:.0f}% [{status}]')

## 3. Optimización de hiperparámetros con Optuna (Phase 3)

Optuna realiza **50 trials** de optimización bayesiana para encontrar
los mejores hiperparámetros de PPO para cada señal.

### Hiperparámetros optimizados (ejemplo)
- Learning rate
- Número de steps por update
- Tamaño de batch
- Número de épocas
- Coeficiente de entropía
- Factor de descuento (gamma)

In [ ]:
# Simulación de optimización Optuna
np.random.seed(42)

n_trials = 50
trials = np.arange(n_trials)

# Simular PFs de Optuna: mejora gradual con ruido
base_pf = 0.8 + 0.6 * (1 - np.exp(-trials / 15)) + np.random.normal(0, 0.15, n_trials)
base_pf = np.clip(base_pf, 0.3, 2.5)
best_so_far = np.maximum.accumulate(base_pf)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# PF por trial
axes[0].scatter(trials, base_pf, alpha=0.6, s=40, color='steelblue', label='PF por trial')
axes[0].plot(trials, best_so_far, color='red', linewidth=2, label='Mejor hasta ahora')
axes[0].axhline(1.0, color='gray', linestyle='--', alpha=0.5)
axes[0].set_xlabel('Trial', fontsize=11)
axes[0].set_ylabel('Profit Factor', fontsize=11)
axes[0].set_title('Optimización Optuna: PF por Trial', fontsize=12)
axes[0].legend()

# Distribución de PFs
axes[1].hist(base_pf, bins=25, color='steelblue', alpha=0.7, edgecolor='white')
axes[1].axvline(base_pf.max(), color='red', linestyle='--', linewidth=2,
                label=f'Mejor: PF={base_pf.max():.2f}')
axes[1].axvline(1.0, color='gray', linestyle='--', alpha=0.5)
axes[1].set_xlabel('Profit Factor', fontsize=11)
axes[1].set_title('Distribución de PF en 50 Trials', fontsize=12)
axes[1].legend()

plt.tight_layout()
plt.show()

print(f'Mejor PF encontrado: {base_pf.max():.2f} (trial {base_pf.argmax()})')
print(f'PF medio: {base_pf.mean():.2f}')
print(f'Trials con PF > 1.0: {(base_pf > 1.0).sum()} / {n_trials}')

## 4. Entrenamiento 30-Fold Ensemble (Phase 4)

### ¿Por qué ensemble de 30 modelos?

Un solo modelo es **frágil**: puede funcionar bien en ciertos periodos y mal en otros.
El ensemble de 30 modelos:

1. **Reduce varianza**: La decisión final es por votación mayoritaria
2. **Mejora robustez**: Cada modelo ve datos ligeramente diferentes
3. **Cuantifica incertidumbre**: La dispersión de votos indica confianza

### Proceso

```
Datos Training (60%)
     │
     ├─ Fold 0: sub-muestra → Modelo 0
     ├─ Fold 1: sub-muestra → Modelo 1
     ├─ ...
     └─ Fold 29: sub-muestra → Modelo 29

Predicción final = Votación mayoritaria de 30 modelos
```

In [ ]:
# Simulación: beneficio del ensemble vs modelo individual
np.random.seed(42)

n_folds = 30
n_decisions = 200  # decisiones de trading

# Cada modelo tiene una precisión individual del ~58%
individual_accuracy = 0.58

# Simular votos de cada modelo
correct_answer = np.random.choice([0, 1], n_decisions)
model_predictions = np.array([
    np.where(np.random.random(n_decisions) < individual_accuracy,
             correct_answer,
             1 - correct_answer)
    for _ in range(n_folds)
])

# Ensemble por votación mayoritaria
ensemble_votes = model_predictions.sum(axis=0)
ensemble_pred = (ensemble_votes > n_folds / 2).astype(int)

# Calcular precisiones
individual_accs = [(model_predictions[i] == correct_answer).mean() for i in range(n_folds)]
ensemble_acc = (ensemble_pred == correct_answer).mean()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Precisión individual vs ensemble
axes[0].bar(range(n_folds), individual_accs, color='lightsteelblue', alpha=0.8, label='Modelo individual')
axes[0].axhline(ensemble_acc, color='red', linewidth=2, linestyle='--',
                label=f'Ensemble ({ensemble_acc:.1%})')
axes[0].axhline(np.mean(individual_accs), color='steelblue', linewidth=1.5, linestyle=':',
                label=f'Media individual ({np.mean(individual_accs):.1%})')
axes[0].set_xlabel('Fold', fontsize=11)
axes[0].set_ylabel('Precisión', fontsize=11)
axes[0].set_title('Precisión: Individual vs Ensemble (30 modelos)', fontsize=12)
axes[0].legend()

# Distribución de votos del ensemble
axes[1].hist(ensemble_votes, bins=range(0, n_folds+2), color='steelblue',
             alpha=0.7, edgecolor='white', align='left')
axes[1].axvline(n_folds/2, color='red', linestyle='--', label='Umbral de decisión (15)')
axes[1].set_xlabel('Votos a favor', fontsize=11)
axes[1].set_ylabel('Frecuencia', fontsize=11)
axes[1].set_title('Distribución de Votos del Ensemble', fontsize=12)
axes[1].legend()

plt.tight_layout()
plt.show()

print(f'Precisión media individual: {np.mean(individual_accs):.1%}')
print(f'Precisión del ensemble:     {ensemble_acc:.1%}')
print(f'Mejora: +{(ensemble_acc - np.mean(individual_accs))*100:.1f} puntos porcentuales')

## 5. Resumen del pipeline de entrenamiento

```
Phase 1: Descubrimiento     → Evaluación estadística de señales
      ↓
Phase 3: Optuna              → 50 trials de hiperparámetros (maximizar PF)
      ↓
Phase 4: 30-Fold Training    → Ensemble de 30 modelos PPO
      ↓
Phase 5: Validación Test     → Evaluación en datos nunca vistos (20%)
```

### Criterios de aprobación

- PF > 1.2 en datos de test (mínimo para cubrir costes)
- Número mínimo de trades (significancia estadística)
- Degradación < 15% entre training y test

### Lecciones aprendidas

1. **La complejidad del modelo no correlaciona con mejores resultados**
2. **El control de rollouts es más importante que la arquitectura**
3. **El ensemble de 30 modelos reduce significativamente la varianza**
4. **SAC fue descartado por no ser reproducible**

> **Siguiente**: `04_backtesting_results.ipynb` — Resultados y métricas de backtesting